#한글용 오픈소스 초거대 언어 모델(Open-Source LLM) 예제 사례

### 작성자: 서진호(jinhoseo@stud.assist.ac.kr)

##### 1. 한글 모델은 이준범님이 공개한 KoAlpaca-Polyglot-12.8B 사용.
##### 2. 주의사항: 반드시 CoPro (유료) 와 Spot Instance 컴퓨팅 단위($9.9) 가입하셔야 코랩에서 NVIDIA A100 사용해서 GPU하고 모델 실행할 수 있음.
##### 3. 대부분 발생하는 에러는 세션이 끊어져서 발생함. 구글 코랩은 전체 사용자의 자원 관리하므로 특정 사용자가 자원을 많이 사용하면 강제로 끊을 수 있음을 유의하기 바람.
##### 4. 전체 다운로드 받는데 빠르면 10분 늦으면 30분 정도 걸림.
##### 5. PyTorch 언어를 사용하고, 8bit로 quantization, 그리고 gradio로 HTML UI 사용

In [ ]:
#오픈소스 실행 전에 반드시 관련 요구 컴포넌트 설치

!pip install -q gradio torch transformers accelerate bitsandbytes Xformers

In [ ]:
import gradio as gr
import torch
from transformers import pipeline, AutoModelForCausalLM
import time


MODEL = "beomi/KoAlpaca-Polyglot-12.8B"
model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    device_map="auto",
    load_in_8bit=True,
    revision="8bit",
    # max_memory=f'{int(torch.cuda.mem_get_info()[0]/1024**3)-2}GB'
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=MODEL,
    # device=2,
)


def answer(state, state_chatbot, text):
    messages = state + [{"role": "질문", "content": text}]

    conversation_history = "\n".join(
        [f"### {msg['role']}:\n{msg['content']}" for msg in messages]
    )

    ans = pipe(
        conversation_history + "\n\n### 답변:",
        do_sample=True,
        max_new_tokens=512,
        temperature=0.5,
        top_p=0.9,
        return_full_text=False,
        eos_token_id=2
    )

    msg = ans[0]["generated_text"]

    if "###" in msg:
        msg = msg.split("###")[0]

    new_state = [{"role": "이전 질문", "content": text}, {"role": "이전 답변", "content": msg}]

    state = state + new_state
    state_chatbot = state_chatbot + [(text, msg)]

    print(state)
    print(state_chatbot)

    return state, state_chatbot, state_chatbot


with gr.Blocks(css="#chatbot .overflow-y-auto{height:750px}") as demo:
    state = gr.State(
        [
            {
                "role": "맥락",
                "content": "KoAlpaca는 EleutherAI에서 개발한 Polyglot-ko 라는 한국어 모델을 기반으로, 자연어 처리 연구자인 이준범인 개발한 모델을 이용했습니다.",
            },
            {
                "role": "맥락",
                "content": "ChatKoAlpaca는 KoAlpaca를 채팅 유형으로 만든 봇입니다.",
            },
            {"role": "명령어", "content": "친절한 AI 챗봇인 ChatKoAlpaca 로서 답변을 합니다."},
            {
                "role": "명령어",
                "content": "인사에는 짧고 간단히 친절하게 인사로 답하고, 아래 대화에 간단하고 짧게 답해줍니다.",
            },
        ]
    )
    state_chatbot = gr.State([])

    with gr.Row():
        gr.HTML(
            """<div style="text-align: center; max-width: 500px; margin: 0 auto;">
            <div>
                <h1>ChatKoAlpaca 12.8B (v1.1b-chat-8bit)</h1>
            </div>
            <div>
                Demo runs on CoPro and Spot Instance(A100) with 8bit quantized
            </div>
        </div>"""
        )

    with gr.Row():
        chatbot = gr.Chatbot(elem_id="chatbot")

    with gr.Row():
        txt = gr.Textbox(show_label=False, placeholder="메시지를 적어주세요...").style(
            container=False
        )

    txt.submit(answer, [state, state_chatbot, txt], [state, state_chatbot, chatbot])
    txt.submit(lambda: "", None, txt)

demo.launch(debug=True, server_name="0.0.0.0")
